In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import io
from google.colab import files
import os
import datetime
import time
import sys
import math
from lxml import etree
from io import StringIO, BytesIO
import seaborn as sns
import yfinance as yf

In [ ]:
!pip install yfinance

     |████████████████████████████████| 5.5MB 6.3MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=f924018751521b79bcc8bbb31cf50afbeeb0b099238c69f4d001ccc82a8ad95d
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# # # !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth 
# from pydrive.drive import GoogleDrive 
# from google.colab import auth 
# from oauth2client.client import GoogleCredentials 
  
  
# # Authenticate and create the PyDrive client. 
# auth.authenticate_user() 
# gauth = GoogleAuth() 
# gauth.credentials = GoogleCredentials.get_application_default() 
# drive = GoogleDrive(gauth)

In [ ]:
#### process all sec submissions dataset to find all 10-Q filings URL

subfolders = []

sec_submissions_folder = os.walk('drive/MyDrive/W266 Final Project/sec_submissions/')
for i in sec_submissions_folder:
    if i[0] != 'drive/MyDrive/W266 Final Project/sec_submissions/':
        subfolders.append(i[0])

In [ ]:
#### concatenate all sec submission datasets (currently broken down quarterly)
sub_datasets = []

for i in subfolders:
    df = pd.read_csv(i + '/sub.tsv', sep = '\t')
    sub_datasets.append(df)
    
all_datasets = pd.concat(sub_datasets, ignore_index=True)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (35,38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
## filter out rows - only keep rows where the form is a 10-Q
ten_q_rows = all_datasets.loc[all_datasets['form'] == '10-Q']

In [ ]:
## load in ticker symbol to CIK mapping dataset
cik_ticker_map = pd.read_csv('drive/MyDrive/W266 Final Project/cik_ticker_sec.txt', sep = '\t', header = None)
cik_ticker_map["ticker"] = cik_ticker_map[0].astype("str").map(lambda x: x.upper())
cik_ticker_map["cik"] = cik_ticker_map[1].astype("int")
cik_ticker_map = cik_ticker_map[["ticker", "cik"]]

## join on 10-Q dataset
ten_q_with_ticker = ten_q_rows.merge(cik_ticker_map, how = 'inner', left_on='cik', right_on='cik')

In [ ]:
## format 10-Q with ticker symbol dataset

## create column with adsh but remove the dashes
ten_q_with_ticker["adsh_nodash"] = ten_q_with_ticker["adsh"].map(lambda x: x.replace("-",""))

## construct the sec URL - this is where all 10-Qs are located as txt files
ten_q_with_ticker['sec_url'] = 'https://www.sec.gov/Archives/edgar/data/' + ten_q_with_ticker["cik"].astype('str') + '/' + ten_q_with_ticker['adsh_nodash'].astype('str') + '/' + ten_q_with_ticker['adsh'].astype('str') + '.txt'

## format filing date and time as datetime object
ten_q_with_ticker["accepted_filing_time"] = ten_q_with_ticker["accepted"].map(lambda x: datetime.datetime.strptime(x[:-2], "%Y-%m-%d %H:%M:%S"))
ten_q_with_ticker["accepted_filing_date"] = ten_q_with_ticker["accepted"].map(lambda x: datetime.datetime.strptime(x[:-2], "%Y-%m-%d %H:%M:%S").date())

## only keep the necessary columns
ten_q_with_ticker = ten_q_with_ticker[["adsh","cik", "ticker", "name", "sec_url", "accepted_filing_time", "accepted_filing_date"]]

In [ ]:
ten_q_with_ticker = ten_q_with_ticker.drop_duplicates().reset_index(drop = True)

In [ ]:
ten_q_with_ticker

,adsh,cik,ticker,name,sec_url,accepted_filing_time,accepted_filing_date
0,0001477932-16-014447,1096275,WKSP,"FRANCHISE HOLDINGS INTERNATIONAL, INC.",https://www.sec.gov/Archives/edgar/data/109627...,2016-12-30 17:42:00,2016-12-30
1,0001477932-16-010684,1096275,WKSP,"FRANCHISE HOLDINGS INTERNATIONAL, INC.",https://www.sec.gov/Archives/edgar/data/109627...,2016-06-02 12:35:00,2016-06-02
2,0001477932-16-012267,1096275,WKSP,"FRANCHISE HOLDINGS INTERNATIONAL, INC.",https://www.sec.gov/Archives/edgar/data/109627...,2016-08-29 06:07:00,2016-08-29
3,0001493152-20-012513,1096275,WKSP,"FRANCHISE HOLDINGS INTERNATIONAL, INC.",https://www.sec.gov/Archives/edgar/data/109627...,2020-07-02 16:25:00,2020-07-02
4,0001493152-20-016843,1096275,WKSP,"WORKSPORT, LTD",https://www.sec.gov/Archives/edgar/data/109627...,2020-08-27 17:03:00,2020-08-27
...,...,...,...,...,...,...,...
81051,0001594062-19-000046,1415952,WLOC,WILLOW CREEK ENTERPRISES INC.,https://www.sec.gov/Archives/edgar/data/141595...,2019-05-23 11:15:00,2019-05-23
81052,0001594062-19-000045,1415952,WLOC,WILLOW CREEK ENTERPRISES INC.,https://www.sec.gov/Archives/edgar/data/141595...,2019-05-23 11:15:00,2019-05-23
81053,0001477932-16-013931,1327238,BNGI,COMPASS BIOTECHNOLOGIES INC.,https://www.sec.gov/Archives/edgar/data/132723...,2016-12-05 15:41:00,2016-12-05
81054,0001213900-19-015882,1771077,PSYC,"TELEMYND, INC.",https://www.sec.gov/Archives/edgar/data/177107...,2019-08-14 17:16:00,2019-08-14


In [ ]:
# ten_q_with_ticker_1 = ten_q_with_ticker[0:10000]
# ten_q_with_ticker_2 = ten_q_with_ticker[10000:20000]
# ten_q_with_ticker_3 = ten_q_with_ticker[20000:30000]
# ten_q_with_ticker_4 = ten_q_with_ticker[30000:40000]
# ten_q_with_ticker_5 = ten_q_with_ticker[40000:50000]
# ten_q_with_ticker_6 = ten_q_with_ticker[50000:60000]
# ten_q_with_ticker_7 = ten_q_with_ticker[60000:68999]


# ten_q_with_ticker_1 = pd.read_csv('drive/My Drive/W266 Final Project/saved_ten_q_data_no_raw_text/ten_q_with_ticker_1.csv')
# ten_q_with_ticker_2 = pd.read_csv('drive/My Drive/W266 Final Project/saved_ten_q_data_no_raw_text/ten_q_with_ticker_2.csv')
# ten_q_with_ticker_3 = pd.read_csv('drive/My Drive/W266 Final Project/saved_ten_q_data_no_raw_text/ten_q_with_ticker_3.csv')
# ten_q_with_ticker_4 = pd.read_csv('drive/My Drive/W266 Final Project/saved_ten_q_data_no_raw_text/ten_q_with_ticker_4.csv')
# ten_q_with_ticker_5 = pd.read_csv('drive/My Drive/W266 Final Project/saved_ten_q_data_no_raw_text/ten_q_with_ticker_5.csv')
# ten_q_with_ticker_6 = pd.read_csv('drive/My Drive/W266 Final Project/saved_ten_q_data_no_raw_text/ten_q_with_ticker_6.csv')
# ten_q_with_ticker_7 = pd.read_csv('drive/My Drive/W266 Final Project/saved_ten_q_data_no_raw_text/ten_q_with_ticker_7.csv')

In [ ]:
# ten_q_with_ticker_1.to_csv('ten_q_with_ticker_1.csv', index = False)
# ten_q_with_ticker_2.to_csv('ten_q_with_ticker_2.csv', index = False)
# ten_q_with_ticker_3.to_csv('ten_q_with_ticker_3.csv', index = False)
# ten_q_with_ticker_4.to_csv('ten_q_with_ticker_4.csv', index = False)
# ten_q_with_ticker_5.to_csv('ten_q_with_ticker_5.csv', index = False)
# ten_q_with_ticker_6.to_csv('ten_q_with_ticker_6.csv', index = False)
# ten_q_with_ticker_7.to_csv('ten_q_with_ticker_7.csv', index = False)


# !cp ten_q_with_ticker_1.csv "drive/My Drive/W266 Final Project/saved_ten_q_data_no_raw_text/"
# !cp ten_q_with_ticker_2.csv "drive/My Drive/W266 Final Project/saved_ten_q_data_no_raw_text/"
# !cp ten_q_with_ticker_3.csv "drive/My Drive/W266 Final Project/saved_ten_q_data_no_raw_text/"
# !cp ten_q_with_ticker_4.csv "drive/My Drive/W266 Final Project/saved_ten_q_data_no_raw_text/"
# !cp ten_q_with_ticker_5.csv "drive/My Drive/W266 Final Project/saved_ten_q_data_no_raw_text/"
# !cp ten_q_with_ticker_6.csv "drive/My Drive/W266 Final Project/saved_ten_q_data_no_raw_text/"
# !cp ten_q_with_ticker_7.csv "drive/My Drive/W266 Final Project/saved_ten_q_data_no_raw_text/"

In [ ]:
# next_dataset_to_process = ten_q_with_ticker_1
# next_dataset_to_process = ten_q_with_ticker_2
# next_dataset_to_process = ten_q_with_ticker_3
# next_dataset_to_process = ten_q_with_ticker_4
# next_dataset_to_process = ten_q_with_ticker_5
# next_dataset_to_process = ten_q_with_ticker_6
# next_dataset_to_process = ten_q_with_ticker_7
# next_dataset_to_process = ten_q_with_ticker_8

In [ ]:
# !cp ten_q_with_ticker_and_raw_text_1_19.csv "drive/My Drive/W266 Final Project/ten_q_data_with_raw_text/"

In [ ]:
ten_q_with_ticker_6 = pd.read_csv('drive/My Drive/W266 Final Project/saved_ten_q_data_no_raw_text/ten_q_with_ticker_6.csv')

In [ ]:
ten_q_with_ticker_6

,adsh,cik,ticker,name,sec_url,accepted_filing_time,accepted_filing_date
0,0001564590-18-027988,750558,QNBC,QNB CORP,https://www.sec.gov/Archives/edgar/data/750558...,2018-11-07 16:03:00,2018-11-07
1,0001564590-19-041102,750558,QNBC,QNB CORP,https://www.sec.gov/Archives/edgar/data/750558...,2019-11-06 16:50:00,2019-11-06
2,0001564590-18-020336,750558,QNBC,QNB CORP,https://www.sec.gov/Archives/edgar/data/750558...,2018-08-08 13:57:00,2018-08-08
3,0001564590-19-017331,750558,QNBC,QNB CORP,https://www.sec.gov/Archives/edgar/data/750558...,2019-05-08 17:16:00,2019-05-08
4,0001564590-16-028346,750558,QNBC,QNB CORP,https://www.sec.gov/Archives/edgar/data/750558...,2016-11-08 16:20:00,2016-11-08
...,...,...,...,...,...,...,...
9995,0001654954-18-008255,1141197,PED,PEDEVCO CORP,https://www.sec.gov/Archives/edgar/data/114119...,2018-07-31 17:20:00,2018-07-31
9996,0001654954-19-006094,1141197,PED,PEDEVCO CORP,https://www.sec.gov/Archives/edgar/data/114119...,2019-05-15 16:06:00,2019-05-15
9997,0001654954-16-003700,1141197,PED,PEDEVCO CORP,https://www.sec.gov/Archives/edgar/data/114119...,2016-11-09 16:44:00,2016-11-09
9998,0001654954-18-005070,1141197,PED,PEDEVCO CORP,https://www.sec.gov/Archives/edgar/data/114119...,2018-05-10 19:56:00,2018-05-10


In [ ]:
def obtain_mda_and_prices_data(df):
    current_set = df
    for index, row in current_set.iterrows():
        try:
              
            starttime = datetime.datetime.now()
            # print("Processing row number", index)
            
            ###### PARSING 10Q SECTION TO EXTRACT MDA ######
            ten_q_raw = requests.get(row["sec_url"]).text

            ## parse the raw text and get rid of tags
            soup = BeautifulSoup(ten_q_raw, 'lxml')
            notags = soup.getText()

            ## use regex to extract MDA
            #### first splitting on all mentions of "Item 2. Management's Discussion and Analysis" ignoring case sensitivity
            #### and taking the last element of split list
            start_of_mda = re.split("Item[\s]*2[\s]*[.\-–:]+[\s]*[.\-–]*[\s]*Management[\S]?s?[\s]*Discussion[\s]*and[\s]*Analysis[\s]*", 
                          notags, 
                          flags = re.IGNORECASE)[-1]
                    
            #### then taking that last element from above and splitting on "Item 3. Quantitative and Qualitative Disclosures About Market Risk" ignoring case sensitivity
            #### and taking the first element of this split list to get the full MDA section
            full_mda = re.split("Item[\s]*[34][\s]*[.\-–:]*[\s]*[.\-–]*[\s]*(Quantitative[\s]*and[\s]*Qualitative[\s]*disclosures[\s]*|Controls[\s]*and[\s]*Procedures[\s]*)", 
                                start_of_mda, 
                                flags = re.IGNORECASE)[0]

            mda_length = len(full_mda)
            full_ten_q_length = len(notags)
            mda_percentage = mda_length/full_ten_q_length

            current_set.loc[index, 'mda_raw_text'] = full_mda
            current_set.loc[index, 'ending_of_mda'] = full_mda[-100:-1]
            current_set.loc[index, 'mda_length'] = mda_length
            current_set.loc[index, 'full_ten_q_length'] = full_ten_q_length
            current_set.loc[index, 'mda_percentage'] = mda_percentage
            endtime = datetime.datetime.now()

            # print("Current row took", endtime - starttime)

            ###### (COMPLETED) PARSING 10Q SECTION TO EXTRACT MDA ######



            ####### OBTAINING STOCK PRICE DATA #######
            #### closing price on filing date and opening price on next trading day
            row_ticker = row["ticker"]
            row_date = datetime.datetime.strptime(row["accepted_filing_date"], "%Y-%m-%d").date() 
            prices_for_timewindow = yf.Ticker(row_ticker).history(start = row_date, end = row_date + datetime.timedelta(days = 6)).reset_index() ## setting time window to 6 days because this will ensure that we get two consecutive trading days (avoiding weekends and holidays)
            prices_for_timewindow = prices_for_timewindow.iloc[:2,:]

            if prices_for_timewindow.shape[0] > 1:          ## only fill in values in columns if pricing data is available for these dates
                current_set.loc[index, "trading_date_closest_to_filing_date"] = prices_for_timewindow.loc[0,"Date"] 
                current_set.loc[index, "closing_price_on_filing_date"] = prices_for_timewindow.loc[0,"Close"]

                current_set.loc[index, "next_trading_date"] = prices_for_timewindow.loc[1,"Date"]
                current_set.loc[index, "opening_price_on_next_trading_date"] = prices_for_timewindow.loc[1,"Open"]


            #### opening price on x days later
            days_after = [7,30]
            for window in days_after:
                prices_for_timewindow = yf.Ticker(row_ticker).history(start = row_date, end = row_date + datetime.timedelta(days = window + 1)).reset_index()
                prices_for_timewindow_rows = prices_for_timewindow.shape[0]

                if prices_for_timewindow_rows > 1:         ## also only fill in values in columns if pricing data is available
                    current_set.loc[index, "trading_date_" + str(window) + "_days_after"] = prices_for_timewindow.loc[prices_for_timewindow_rows - 1,"Date"]
                    current_set.loc[index, "opening_price_" + str(window) + "_days_after"] = prices_for_timewindow.loc[prices_for_timewindow_rows - 1,"Open"]

            ####### (COMPLETED) OBTAINING STOCK PRICE DATA #######


            # no need to sleep because each iteration takes about 1.5 seconds (and the limit is 10 calls per second)
            time.sleep(0.1)
            if index%100 == 0:
                print("Just finished row number", index)
                print(datetime.datetime.now())
                print()
        except:
            print("Hit an error while processing row", str(index))
            print("Continuing to next row")
            continue

    return current_set

In [ ]:
def chunk_process_data_and_write(df, subset, chunks):
    rows_in_df = df.shape[0]
    chunk_size = math.floor(rows_in_df / chunks)

    for i in range(chunks):
        if i != chunks - 1:
            starting = i * chunk_size
            ending = chunk_size * (i+1)
        else:
            starting = i * chunk_size
            ending = -1

        print("Processing Chunk:", starting, "to", ending)

        next_dataset_to_process = df[starting:ending]
        output = obtain_mda_and_prices_data(next_dataset_to_process)
        
        print("Writing to CSV: " + 'data_augmented_' + str(subset) + '_' + str(starting) + 'to' + str(ending) + '.csv')
        output.to_csv('drive/MyDrive/W266 Final Project/ten_q_with_mda/data_augmented_' + str(subset) + '_' + str(starting) + 'to' + str(ending) + '.csv', index = False)

In [ ]:
chunk_process_data_and_write(ten_q_with_ticker_6[5000:], 6666, 5)

Processing Chunk: 0 to 1000
Just finished row number 5000
2021-02-21 20:30:11.236922

- DCP-PC: Data doesn't exist for startDate = 1494374400, endDate = 1494892800
- DCP-PC: Data doesn't exist for startDate = 1494374400, endDate = 1495065600
- DCP-PC: Data doesn't exist for startDate = 1494374400, endDate = 1497052800
- DCP-PB: Data doesn't exist for startDate = 1494374400, endDate = 1494892800
- DCP-PB: Data doesn't exist for startDate = 1494374400, endDate = 1495065600
- DCP-PB: Data doesn't exist for startDate = 1494374400, endDate = 1497052800
- DCP-PC: Data doesn't exist for startDate = 1533686400, endDate = 1534204800
- DCP-PC: Data doesn't exist for startDate = 1533686400, endDate = 1534377600
- DCP-PC: Data doesn't exist for startDate = 1533686400, endDate = 1536364800
- DCP-PC: Data doesn't exist for startDate = 1478131200, endDate = 1478649600
- DCP-PC: Data doesn't exist for startDate = 1478131200, endDate = 1478822400
- DCP-PC: Data doesn't exist for startDate = 1478131200,

In [ ]:
test = pd.read_csv('/content/data_augmented_2_3000to4000.csv')

In [ ]:
test.loc[test["closing_price_on_filing_date"].isnull(),:]

,adsh,cik,ticker,name,sec_url,accepted_filing_time,accepted_filing_date,mda_raw_text,ending_of_mda,mda_length,full_ten_q_length,mda_percentage,trading_date_closest_to_filing_date,closing_price_on_filing_date,next_trading_date,opening_price_on_next_trading_date,trading_date_7_days_after,opening_price_7_days_after,trading_date_30_days_after,opening_price_30_days_after
62,0000039263-17-000033,39263,CFR-PB,"CULLEN/FROST BANKERS, INC.",https://www.sec.gov/Archives/edgar/data/39263/...,2017-07-27 11:55:00,2017-07-27,of Financial Condition and Results of Operatio...,"not significant, and, as such, separate quanti...",5320.0,6182523.0,0.000860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,0000039263-17-000039,39263,CFR-PB,"CULLEN/FROST BANKERS, INC.",https://www.sec.gov/Archives/edgar/data/39263/...,2017-10-26 12:19:00,2017-10-26,of Financial Condition and Results of Operatio...,"not significant, and, as such, separate quanti...",5478.0,6229701.0,0.000879,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,0000039263-20-000027,39263,CFR-PB,"CULLEN/FROST BANKERS, INC.",https://www.sec.gov/Archives/edgar/data/39263/...,2020-04-30 14:42:00,2020-04-30,of Financial Condition and Results of Operatio...,ix:\nus-gaap_\n\n\n Data Type:\nxbrli:stringIt...,7137504.0,7507091.0,0.950768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,0000039263-16-000079,39263,CFR-PB,"CULLEN/FROST BANKERS, INC.",https://www.sec.gov/Archives/edgar/data/39263/...,2016-04-27 12:48:00,2016-04-27,of Financial Condition and Results of Operatio...,"not significant, and, as such, separate quanti...",5217.0,5458898.0,0.000956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,0000039263-16-000091,39263,CFR-PB,"CULLEN/FROST BANKERS, INC.",https://www.sec.gov/Archives/edgar/data/39263/...,2016-07-27 12:03:00,2016-07-27,of Financial Condition and Results of Operatio...,"not significant, and, as such, separate quanti...",5148.0,6153618.0,0.000837,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
917,0001004980-18-000005,1004980,PCGU,PG&E CORP,https://www.sec.gov/Archives/edgar/data/100498...,2018-05-03 14:51:00,2018-05-03,OF FINANCIAL CONDITION AND RESULTS OF OPERATIO...,the “News & Events: Events & Presentations” t...,108163.0,2935988.0,0.036840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
921,0001004980-19-000028,1004980,PCG-PC,PG&E CORP,https://www.sec.gov/Archives/edgar/data/100498...,2019-08-09 16:52:00,2019-08-09,OF FINANCIAL CONDITION AND RESULTS OF OPERATIO...,website solely for the information of investo...,157576.0,5499882.0,0.028651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
925,0001004980-19-000028,1004980,PCG-PI,PG&E CORP,https://www.sec.gov/Archives/edgar/data/100498...,2019-08-09 16:52:00,2019-08-09,OF FINANCIAL CONDITION AND RESULTS OF OPERATIO...,website solely for the information of investo...,157576.0,5499882.0,0.028651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
927,0001004980-19-000028,1004980,PCGU,PG&E CORP,https://www.sec.gov/Archives/edgar/data/100498...,2019-08-09 16:52:00,2019-08-09,OF FINANCIAL CONDITION AND RESULTS OF OPERATIO...,website solely for the information of investo...,157576.0,5499882.0,0.028651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Item[\s]*2[\s]*[.\-–:]+[\s]*[.\-–]*[\s]*Management[\S]?s[\s]*Discussion[\s]*and[\s]*Analysis[\s]*
# Item[\s]*[34][\s]*[.\-–:]*[\s]*[.\-–]*[\s]*(Quantitative[\s]*and[\s]*Qualitative[\s]*disclosures[\s]*|Controls[\s]*and[\s]*Procedures[\s]*)

In [ ]:
##### REGEX ALREADY INCLUDES

# Item 2.Management’s Discussion and Analysis of Financial Condition and Results of Operations.IntroductionThis MDA provides additional information

# Item 2. Managements Discussion and Analysis of Financial
# Condition and Results of Operations.

# Item 2.Management’s Discussion and Analysis of Financial Condition and Results of Operations Forward-lookin

# ITEM 2.Management’s Discussion and Analysis of Financial Condition and Results of Operations

# Item 2 – MANAGEMENT’S DISCUSSION
# AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS.

# ITEM 2.   MANAGEMENT’S
# DISCUSSION AND ANALYSIS

# Item
# 2.     Management’s Discussion and Analysis or Plan of Operation

# ITEM
# 2. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITIONS AND RESULTS OF OPERATIONS


######## FOR ENDING
# ITEM
# 3 – QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK

# ITEM 3.  QUANTITATIVE AND QUALITATIVE DISCLOSURES

# ITEM
# 3	QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK 

# ITEM 3.   QUANTITATIVE AND QUALITATIVE DISCLOSURES
# ABOUT MARKET RISK.

# ITEM 3.   QUANTITATIVE AND QUALITATIVE DISCLOSURES
# ABOUT MARKET RISK.

# Item 3 Quantitative and Qualitative Disclosures About

# ITEM 4. CONTROLS AND PROCEDURES

# ITEM 4. CONTROLS AND PROCEDURES.

# ITEM 3 . QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET 

# ITEM 3–QUANTITATIVE AND QUALITATIVE
# DISCLOSURES ABOUT MARKET RISK

# ITEM 3. - QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK

# ITEM 3. - QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK

# ITEM 3 – QUANTITATIVE AND QUALITATIVE
# DISCLOSURES ABOUT MARKET RISK

# ITEM 3:QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT
#                                          MARKET RISK



##### REGEX STILL NEEDS TO INCLUDE



##### REGEX MUST EXCLUDE
# ITEM 2. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS - continued